# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
from datetime import timedelta
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Format today and yesterday into strings for filename creation
format = "%m-%d-%Y"
now = datetime.now()
today = now.strftime(format)
yesterday = (now - timedelta(days=1)).strftime(format)

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    jh_live_date = today
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    jh_live_date = yesterday
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
print(jh_live_date)

01-20-2023


In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2023-01-21 04:20:55,33.93911,67.709953,208084,7864,NaN,NaN,Afghanistan,534.530870,3.779243
1,NaN,NaN,NaN,Albania,2023-01-21 04:20:55,41.15330,20.168300,334084,3596,NaN,NaN,Albania,11609.006880,1.076376
2,NaN,NaN,NaN,Algeria,2023-01-21 04:20:55,28.03390,1.659600,271316,6881,NaN,NaN,Algeria,618.721885,2.536157
3,NaN,NaN,NaN,Andorra,2023-01-21 04:20:55,42.50630,1.521800,47820,165,NaN,NaN,Andorra,61890.894972,0.345044
4,NaN,NaN,NaN,Angola,2023-01-21 04:20:55,-11.20270,17.873900,105095,1930,NaN,NaN,Angola,319.765542,1.836434


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/11/23,1/12/23,1/13/23,1/14/23,1/15/23,1/16/23,1/17/23,1/18/23,1/19/23,1/20/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,207900,207900,207900,207900,207900,207993,208009,208034,208062,208084
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,333995,334018,334018,334029,334037,334037,334064,334084,334084,334084
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,271268,271277,271286,271287,271287,271287,271292,271296,271307,271316


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/11/23,1/12/23,1/13/23,1/14/23,1/15/23,1/16/23,1/17/23,1/18/23,1/19/23,1/20/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,7854,7854,7854,7854,7854,7857,7859,7860,7864,7864
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3596,3596,3596,3596,3596,3596,3596,3596,3596,3596
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,6881,6881,6881,6881,6881,6881,6881,6881,6881,6881


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/11/23,1/12/23,1/13/23,1/14/23,1/15/23,1/16/23,1/17/23,1/18/23,1/19/23,1/20/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/11/23,1/12/23,1/13/23,1/14/23,1/15/23,1/16/23,1/17/23,1/18/23,1/19/23,1/20/23
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,19318,19318,19318,19318,19318,19318,19318,19389,19389,19389
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,68518,68518,68518,68518,68518,68518,68518,68764,68764,68764
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,7188,7188,7188,7188,7188,7188,7188,7258,7258,7258


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/11/23,1/12/23,1/13/23,1/14/23,1/15/23,1/16/23,1/17/23,1/18/23,1/19/23,1/20/23
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,230,230,230,230,230,230,230,230,230,230
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,721,721,721,721,721,721,721,722,722,722
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,103,103,103,103,103,103,103,103,103,103


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
678,1001.0,Autauga,Alabama,US,2023-01-21 04:20:55,32.539527,-86.644082,19389,230,NaN,NaN,"Autauga, Alabama, US",34704.397788,1.186240
715,1075.0,Lamar,Alabama,US,2023-01-21 04:20:55,33.779950,-88.096680,4635,73,NaN,NaN,"Lamar, Alabama, US",33574.791742,1.574973
716,1077.0,Lauderdale,Alabama,US,2023-01-21 04:20:55,34.901719,-87.656247,29801,418,NaN,NaN,"Lauderdale, Alabama, US",32137.734689,1.402637


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,101998621,1104103,0.0
India,44681650,530728,0.0
France,39681509,164793,0.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,101998621,1104103,0.0,100894518.0,2023-01-21 04:20:55,37.934269,-91.444083
India,44681650,530728,0.0,44150922.0,2023-01-21 04:20:55,23.088275,81.806127
France,39681509,164793,0.0,39516716.0,2023-01-21 04:20:55,6.430808,-34.730285
Germany,37668384,164703,0.0,37503681.0,2023-01-21 04:20:55,51.571844,10.277881
Brazil,36712851,696188,0.0,36016663.0,2023-01-21 04:20:55,-12.669522,-48.480493


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,11957408,99685,0.0
Texas,8329536,92367,0.0
Florida,7443954,84927,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,11957408,99685,0.0,11857723.0,2023-01-21 04:20:55,37.843962,-120.728594
Texas,8329536,92367,0.0,8237169.0,2023-01-21 04:20:55,31.660643,-98.653069
Florida,7443954,84927,0.0,7359027.0,2023-01-21 04:20:55,28.940755,-82.700744
New York,6672225,75945,0.0,6596280.0,2023-01-21 04:20:55,42.544151,-75.474183
Illinois,4008844,40980,0.0,3967864.0,2023-01-21 04:20:55,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,3665365,35079,0.0
Florida,Miami-Dade,1524998,12049,0.0
Illinois,Cook,1502422,15127,0.0
Arizona,Maricopa,1493595,18591,0.0
Texas,Harris,1255228,11495,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,3665365,35079,0.0,3630286.0,2023-01-21 04:20:55,34.308284,-118.228241,6037.0
Florida,Miami-Dade,1524998,12049,0.0,1512949.0,2023-01-21 04:20:55,25.611236,-80.551706,12086.0
Illinois,Cook,1502422,15127,0.0,1487295.0,2023-01-21 04:20:55,41.841448,-87.816588,17031.0
Arizona,Maricopa,1493595,18591,0.0,1475004.0,2023-01-21 04:20:55,33.348359,-112.491815,4013.0
Texas,Harris,1255228,11495,0.0,1243733.0,2023-01-21 04:20:55,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2023-01-18,208034,334084,271296,47781,105095,11,9106,10024095,446008,11247412,...,1029564,250307,12014,551062,11526254,703228,535,11945,338090,259981
2023-01-19,208062,334084,271307,47820,105095,11,9106,10024095,446008,11265457,...,1029564,250332,12014,551191,11526284,703228,535,11945,338407,259981
2023-01-20,208084,334084,271316,47820,105095,11,9106,10024095,446008,11274262,...,1029564,250360,12014,551191,11526301,703228,535,11945,338656,259981


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2023-01-18,7860,3596,6881,165,1930,0,146,130338,8717,17712,...,7597,1637,14,5837,43186,5708,0,2159,4037,5637
2023-01-19,7864,3596,6881,165,1930,0,146,130338,8717,18023,...,7597,1637,14,5840,43186,5708,0,2159,4037,5637
2023-01-20,7864,3596,6881,165,1930,0,146,130338,8717,18092,...,7597,1637,14,5840,43186,5708,0,2159,4038,5637


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2023-01-18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-01-19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-01-20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/11/23,1/12/23,1/13/23,1/14/23,1/15/23,1/16/23,1/17/23,1/18/23,1/19/23,1/20/23
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,19318,19318,19318,19318,19318,19318,19318,19389,19389,19389
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,4626,4626,4626,4626,4626,4626,4626,4635,4635,4635
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,29636,29636,29636,29636,29636,29636,29636,29801,29801,29801
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,9210,9210,9210,9210,9210,9210,9210,9243,9243,9243
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,46236,46236,46236,46236,46236,46236,46236,46420,46420,46420


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2023-01-18,1602891,302921,8309,2394646,992745,11945805,1743671,960940,324137,49,...,2460056,8326505,1079001,149687,24176,2240431,1899401,631197,1975535,183586
2023-01-19,1602891,302921,8309,2394646,993143,11952683,1743671,961676,325083,49,...,2460056,8327589,1080067,149687,24228,2240431,1899401,631504,1976404,183586
2023-01-20,1602891,302921,8317,2394646,993713,11957408,1744018,962247,325083,49,...,2460056,8329536,1080067,149687,24269,2240431,1899401,631811,1977071,183586


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2023-01-18       19389   68764    7258  7889  18130    2956   6347   40377   
2023-01-19       19389   68764    7258  7889  18130    2956   6347   40377   
2023-01-20       19389   68764    7258  7889  18130    2956   6347   40377   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2023-01-18        10599     6628  ...    7643   2254     9774     2303   
2023-01-19        10599     6628  ...    7643   2254     9774     2303   
2023-01-20        10599     6628  ...    7643   2254     9774     2303   

Province_State                                                     
Admin2         Sweetwater  Teton Uinta Unassigned Washakie Weston  
2023-01-18          12442  12065  6346          0     2733   1884  
2023-01-19          12442  12065  6346          0     2733   1884  
2023-01-20          12442  12065  6346          0     2733   1884  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2023-01-18,20846,1455,34,32631,12766,99422,13985,11895,3220,0,...,28801,92363,5222,884,129,22962,15312,7790,15989,1970
2023-01-19,20846,1455,34,32631,12774,99685,13985,11899,3243,0,...,28801,92365,5234,884,129,22962,15312,7790,15999,1970
2023-01-20,20846,1455,34,32631,12781,99685,13985,11903,3243,0,...,28801,92367,5234,884,129,22962,15312,7802,16006,1970


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2023-01-18         230     722     103  109    261      54    130     672   
2023-01-19         230     722     103  109    261      54    130     672   
2023-01-20         230     722     103  109    261      54    130     672   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2023-01-18          170       89  ...     152     44       87       28   
2023-01-19          170       89  ...     152     44       87       28   
2023-01-20          170       89  ...     152     44       87       28   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2023-01-18            137    16    43          0       47     22  
2023-01-19            137    16    43          0       47     22  
2023-01-20            137    16    43          0       47     22  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change().round(7)
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2023-01-18,0.000120,0.00006,0.000015,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000136,0.0,0.000069,0.000004,0.0,0.0,0.0,0.000918,0.0
2023-01-19,0.000135,0.00000,0.000041,0.000816,0.0,0.0,0.0,0.0,0.0,0.001604,...,0.0,0.000100,0.0,0.000234,0.000003,0.0,0.0,0.0,0.000938,0.0
2023-01-20,0.000106,0.00000,0.000033,0.000000,0.0,0.0,0.0,0.0,0.0,0.000782,...,0.0,0.000112,0.0,0.000000,0.000002,0.0,0.0,0.0,0.000736,0.0


In [60]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [61]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change().round(7)
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2023-01-18,0.000127,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,0.0,0.000000,0.0
2023-01-19,0.000509,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.017559,...,0.0,0.0,0.0,0.000514,0.0,0.0,NaN,0.0,0.000000,0.0
2023-01-20,0.000000,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.003828,...,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,0.0,0.000248,0.0


In [62]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [63]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change().round(7)
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2023-01-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [65]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change().round(7)
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2023-01-18,0.004224,0.0,0.000000,0.00115,0.000463,0.000389,0.002240,0.000919,0.000000,0.0,...,0.0,0.002729,0.000000,0.002606,0.001574,0.0,0.00243,0.000617,0.000387,0.0
2023-01-19,0.000000,0.0,0.000000,0.00000,0.000401,0.000576,0.000000,0.000766,0.002919,0.0,...,0.0,0.000130,0.000988,0.000000,0.002151,0.0,0.00000,0.000486,0.000440,0.0
2023-01-20,0.000000,0.0,0.000963,0.00000,0.000574,0.000395,0.000199,0.000594,0.000000,0.0,...,0.0,0.000234,0.000000,0.000000,0.001692,0.0,0.00000,0.000486,0.000338,0.0


In [66]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [67]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change().round(7)
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                   \
Admin2           Autauga  Baldwin   Barbour      Bibb    Blount   Bullock   
2023-01-18      0.003675  0.00359  0.009739  0.004328  0.004043  0.005784   
2023-01-19      0.000000  0.00000  0.000000  0.000000  0.000000  0.000000   
2023-01-20      0.000000  0.00000  0.000000  0.000000  0.000000  0.000000   

Province_State                                         ... Wyoming         \
Admin2           Butler   Calhoun  Chambers  Cherokee  ...    Park Platte   
2023-01-18      0.00459  0.004578  0.005121  0.005766  ...     0.0    0.0   
2023-01-19      0.00000  0.000000  0.000000  0.000000  ...     0.0    0.0   
2023-01-20      0.00000  0.000000  0.000000  0.000000  ...     0.0    0.0   

Province_State                                                               \
Admin2         Sheridan Sublette Sweetwater Teton Uinta Unassigned Washakie   
2023-01-18          0.0      0.0        0.0   0.0   0.0        NaN      0.0   
2023-01-19          0.0      0.0        0.0   0.0   0.0        NaN      0.0   
2023-01-20          0.0      0.0        0.0   0.0   0.0        NaN      0.0   

Province_State         
Admin2         Weston  
2023-01-18        0.0  
2023-01-19        0.0  
2023-01-20        0.0  

[3 rows x 3336 columns]

In [68]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [69]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change().round(7)
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2023-01-18,0.001249,0.0,0.0,0.003938,0.000549,0.001037,0.00366,0.002613,0.000000,NaN,...,0.0,0.002747,0.000000,0.003405,0.0,0.0,0.005714,0.000642,0.000438,0.0
2023-01-19,0.000000,0.0,0.0,0.000000,0.000627,0.002645,0.00000,0.000336,0.007143,NaN,...,0.0,0.000022,0.002298,0.000000,0.0,0.0,0.000000,0.000000,0.000625,0.0
2023-01-20,0.000000,0.0,0.0,0.000000,0.000548,0.000000,0.00000,0.000336,0.000000,NaN,...,0.0,0.000022,0.000000,0.000000,0.0,0.0,0.000000,0.001540,0.000438,0.0


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change().round(7)
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                    \
Admin2         Autauga   Baldwin Barbour      Bibb Blount Bullock Butler   
2023-01-18         0.0  0.001387     0.0  0.009259    0.0     0.0    0.0   
2023-01-19         0.0  0.000000     0.0  0.000000    0.0     0.0    0.0   
2023-01-20         0.0  0.000000     0.0  0.000000    0.0     0.0    0.0   

Province_State                             ... Wyoming                  \
Admin2          Calhoun Chambers Cherokee  ...    Park Platte Sheridan   
2023-01-18      0.00149      0.0      0.0  ...     0.0    0.0      0.0   
2023-01-19      0.00000      0.0      0.0  ...     0.0    0.0      0.0   
2023-01-20      0.00000      0.0      0.0  ...     0.0    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2023-01-18          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2023-01-19          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2023-01-20          0.0        0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [72]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [73]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean().round(7)
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2023-01-18,0.000136,0.000054,0.000013,0.000000,0.0,0.0,0.0,0.000247,0.000009,0.000030,...,0.000463,0.000144,0.0,0.000051,0.000003,0.0,0.0,0.0,0.000912,0.0
2023-01-19,0.000135,0.000027,0.000027,0.000408,0.0,0.0,0.0,0.000123,0.000005,0.000817,...,0.000231,0.000122,0.0,0.000143,0.000003,0.0,0.0,0.0,0.000925,0.0
2023-01-20,0.000121,0.000013,0.000030,0.000204,0.0,0.0,0.0,0.000062,0.000002,0.000799,...,0.000116,0.000117,0.0,0.000071,0.000002,0.0,0.0,0.0,0.000830,0.0


In [74]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [75]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2023-01-18,0.000175,0.0,0.0,0.0,0.0,NaN,0.0,0.000086,0.000014,0.000162,...,0.000183,0.0,0.0,0.000016,0.0,0.0,NaN,0.0,0.000089,0.0
2023-01-19,0.000342,0.0,0.0,0.0,0.0,NaN,0.0,0.000043,0.000007,0.008860,...,0.000092,0.0,0.0,0.000265,0.0,0.0,NaN,0.0,0.000044,0.0
2023-01-20,0.000171,0.0,0.0,0.0,0.0,NaN,0.0,0.000022,0.000004,0.006344,...,0.000046,0.0,0.0,0.000133,0.0,0.0,NaN,0.0,0.000146,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [77]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean().round(7)
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2023-01-18,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2023-01-19,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2023-01-20,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078


In [78]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [79]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean().round(7)
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2023-01-18,0.002134,0.000540,0.000007,0.000587,0.000365,0.000462,0.000748,0.000642,0.000012,0.0,...,0.000024,0.001423,0.000014,0.001315,0.001940,0.001199,0.001227,0.000677,0.000383,0.000276
2023-01-19,0.001067,0.000270,0.000003,0.000294,0.000383,0.000519,0.000374,0.000704,0.001465,0.0,...,0.000012,0.000776,0.000501,0.000657,0.002045,0.000600,0.000613,0.000581,0.000411,0.000138
2023-01-20,0.000534,0.000135,0.000483,0.000147,0.000478,0.000457,0.000287,0.000649,0.000733,0.0,...,0.000006,0.000505,0.000251,0.000329,0.001869,0.000300,0.000307,0.000534,0.000374,0.000069


In [80]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [81]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean().round(7)
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2023-01-18      0.001861  0.001815  0.004907  0.002188  0.002045  0.002910   
2023-01-19      0.000931  0.000907  0.002453  0.001094  0.001022  0.001455   
2023-01-20      0.000465  0.000454  0.001227  0.000547  0.000511  0.000727   

Province_State                                         ...   Wyoming  \
Admin2           Butler   Calhoun  Chambers  Cherokee  ...      Park   
2023-01-18      0.00232  0.002318  0.002593  0.002915  ...  0.000297   
2023-01-19      0.00116  0.001159  0.001297  0.001458  ...  0.000149   
2023-01-20      0.00058  0.000580  0.000648  0.000729  ...  0.000074   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2023-01-18      0.000005  0.000234  0.000217   0.000102  0.000417  0.000518   
2023-01-19      0.000003  0.000117  0.000109   0.000051  0.000208  0.000259   
2023-01-20      0.000001  0.000058  0.000054   0.000025  0.000104  0.000130   

Province_State                               
Admin2         Unassigned  Washakie  Weston  
2023-01-18      -0.503937  0.000186  0.0004  
2023-01-19      -0.503937  0.000093  0.0002  
2023-01-20      -0.503937  0.000046  0.0001  

[3 rows x 3336 columns]

In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [83]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2023-01-18,0.000633,0.0,0.0,0.001992,0.000313,0.000557,0.001624,0.001320,0.000003,NaN,...,0.000031,0.001415,0.000024,0.002147,0.002076,0.001545,0.002884,0.000593,0.000479,0.000390
2023-01-19,0.000316,0.0,0.0,0.000996,0.000470,0.001601,0.000812,0.000828,0.003573,NaN,...,0.000015,0.000718,0.001161,0.001074,0.001038,0.000773,0.001442,0.000296,0.000552,0.000195
2023-01-20,0.000158,0.0,0.0,0.000498,0.000509,0.000801,0.000406,0.000582,0.001786,NaN,...,0.000008,0.000370,0.000581,0.000537,0.000519,0.000386,0.000721,0.000918,0.000495,0.000098


In [84]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [85]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State Alabama                                                       \
Admin2         Autauga   Baldwin Barbour      Bibb    Blount Bullock Butler   
2023-01-18         0.0  0.000704     0.0  0.004630  0.000015     0.0    0.0   
2023-01-19         0.0  0.000352     0.0  0.002315  0.000008     0.0    0.0   
2023-01-20         0.0  0.000176     0.0  0.001157  0.000004     0.0    0.0   

Province_State                              ...   Wyoming                   \
Admin2           Calhoun Chambers Cherokee  ...      Park Platte  Sheridan   
2023-01-18      0.000763      0.0      0.0  ...  0.001656    0.0  0.000023   
2023-01-19      0.000381      0.0      0.0  ...  0.000828    0.0  0.000011   
2023-01-20      0.000191      0.0      0.0  ...  0.000414    0.0  0.000006   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2023-01-18          0.0   0.001838   0.0   0.0       -1.0      0.0    0.0  
2023-01-19          0.0   0.000919   0.0   0.0       -1.0      0.0    0.0  
2023-01-20          0.0   0.000460   0.0   0.0       -1.0      0.0    0.0  

[3 rows x 3336 columns]

In [86]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [87]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

HTTPError: HTTP Error 403: Forbidden

In [ ]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [ ]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

In [ ]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [ ]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

In [ ]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

#### 